In [25]:
# general packages
import numpy as np
import pandas as pd
import datatable as dt
import gc

#model
from sklearn.ensemble import RandomForestClassifier

# Tools and metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from statlearning import plot_feature_importance
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [12]:
%%time
data = pd.read_csv('features_processed.csv') 

Wall time: 58 s


In [13]:
data = data.iloc[:,1:]

In [15]:
data.iloc[:,:]

,Conditions,AreaNumber,PuLat,PuLong,create_by_hour,request_by_hour,pick,BookingSource_Internet,BookingSource_Permanent,BookingSource_StratumIVR,...,create_part_of_day_Afternoon,create_part_of_day_Evening,create_part_of_day_Morning,create_part_of_day_Night,request_part_of_day_Afternoon,request_part_of_day_Evening,request_part_of_day_Morning,request_part_of_day_Night,BookingisDispatch_False,BookingisDispatch_True
0,0,177,-33.921022,151.244202,10,11,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1,0,111,-33.755931,151.273028,9,9,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
2,0,92,-33.897201,151.261205,0,0,1,1,0,0,...,0,0,0,1,0,0,0,1,0,1
3,0,182,-33.895716,151.228609,9,9,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
4,0,4,-33.859737,151.213307,21,21,1,1,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3299995,0,265,-33.827898,151.126799,20,6,1,1,0,0,...,0,1,0,0,0,0,1,0,1,0
3299996,0,149,-33.712023,151.095004,17,17,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
3299997,0,219,-33.850450,151.134428,8,8,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
3299998,0,177,-33.915317,151.251008,8,8,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [16]:
data.dtypes.head(10)

Conditions                    int64
AreaNumber                    int64
PuLat                       float64
PuLong                      float64
create_by_hour                int64
request_by_hour               int64
pick                          int64
BookingSource_Internet        int64
BookingSource_Permanent       int64
BookingSource_StratumIVR      int64
dtype: object

## Training

In [17]:
training_data = data.iloc[:3000000,:]
validation_data = data.iloc[3000000:,:]

In [18]:
y_train = training_data[['TargetVariable_Cancelled','TargetVariable_Completed','TargetVariable_ELSE','TargetVariable_No Job']]
X_train = training_data.drop(columns=['TargetVariable_Cancelled','TargetVariable_Completed','TargetVariable_ELSE','TargetVariable_No Job'])
y_val = validation_data[['TargetVariable_Cancelled','TargetVariable_Completed','TargetVariable_ELSE','TargetVariable_No Job']]
X_val = validation_data.drop(columns=['TargetVariable_Cancelled','TargetVariable_Completed','TargetVariable_ELSE','TargetVariable_No Job'])

## Modeling

Setup cross validation and define error metrics

### Random Forest

In [26]:
gc.collect()

415

In [27]:
%%time
rf = RandomForestClassifier()
tuning_parameters = {
    'n_estimators' : [50],
     'max_depth' :[4],
     'min_samples_split' :[6],
    'min_samples_leaf' :[2]
}

rf = RandomizedSearchCV(rf, tuning_parameters, cv = 5, n_jobs=4) #n_iter = 32,
rf_full_data = rf.fit(X_train, y_train)
rf.fit(X_train, y_train)

print('Best parameters found by grid search:', rf.best_params_, '\n')

C:\Users\1997a\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:289: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


MemoryError: Unable to allocate 1.02 GiB for an array with shape (57, 2400000) and data type int64

In [24]:
# feature importance
plot_feature_importance(rf, X_train.columns)
plt.show()

AttributeError: 'RandomizedSearchCV' object has no attribute 'feature_importances_'

In [ ]:
# Initialise table
columns=['accuracy', 'f1_score']
rows=['RandomForest']
results =pd.DataFrame(0.0, columns=columns, index=rows)

In [ ]:
# Computer metrics
results.iloc[0, 0] = accuracy_score(rf.predict(X_train), y_train)
results.iloc[0, 1] = f1_score(rf.predict(X_train), y_train)
results.round(4)